In [1]:
import h5py
import numpy as np

In [2]:
from dynamics import explicitCorrectionFactorStokesForce, explicitCorrectionFactorTorque, shapeFactor, translationalResistanceCoefficients, resistanceCoefficient

def hopfBifurcation(beta, curly_R, curly_V):
    A_perp, A_para = translationalResistanceCoefficients(beta)
    gamma = np.log(beta + np.sqrt(beta ** 2 - 1 + 0j)) / (beta * np.sqrt(beta ** 2 - 1 + 0j))
    gamma = np.real(gamma)
    C_perp = + 8 * beta * (beta ** 4 - 1) \
        / (9 * beta ** 2 * ((2 * beta ** 2 - 1) * gamma - 1))
    C_para = - 8 * beta * (beta ** 2 - 1) \
        / (9 * (gamma - 1) * beta ** 2)
    F_lambda = shapeFactor(beta)
    A_g = resistanceCoefficient(beta, A_perp, A_para)

    Re_p0 = curly_R * curly_V / (6 * np.pi)
    C_F = explicitCorrectionFactorStokesForce(Re_p0=Re_p0, beta=beta)
    Re_p = max(1, beta) * 4 / (3 * A_g) * (np.sqrt(1 + (3 / 2) * Re_p0 * C_F) - 1) / C_F
    C_T = explicitCorrectionFactorTorque(Re_p, beta, F_lambda)


    curly_A_F = C_F * curly_R * curly_V / (32 * np.pi)
    curly_A_T = C_T * beta * curly_R ** 3 * curly_V ** 2 * F_lambda * max(1, beta) ** 3 / (972 * np.pi)
    v_g = (np.sqrt(1 + 8 * curly_A_F) - 1) / (4 * A_perp * curly_A_F)

    return (
        A_para*C_perp*beta*(1+3*A_para*curly_A_F*v_g-A_perp*curly_A_F*v_g)
        *(5*C_perp*beta+3*A_para**2*curly_A_F*(1+beta**2)*v_g-A_para*(1+beta**2)*(-1+A_perp*curly_A_F*v_g))
    ) / (v_g**2*(
        10*C_perp*beta-A_perp*(1+beta**2)
        +6*A_para**2*curly_A_F*(1+beta**2)*v_g
        -2*A_perp**2*curly_A_F*(1+beta**2)*v_g
        -2*A_para*(1+beta**2)*(-1+A_perp*curly_A_F*v_g)
    )) - curly_A_T


In [ ]:
from scipy.optimize import root_scalar
curly_R = 1.2e3 / 1.204
with h5py.File("parameter_scan_001.h5") as f:
    betas = f['betas'][:]
curly_Vs = []
for beta in betas:
    curly_Vs.append(root_scalar(
        lambda x: hopfBifurcation(beta, curly_R, x),
        x0=9.81 * 1e-9 / 1.51147e-5 ** 2
    ).root)
curly_Vs = np.array(curly_Vs)

In [ ]:
import matplotlib.pyplot as plt
volumes = np.zeros(21)
with h5py.File("parameter_scan_001.h5") as f:
    for i in range(21):
        volumes[i] = f[f'{i}/particle_volume'][()]
    betas = f['betas'][:]
# TODO(Nils); Implement theoretical formula. Calculate shape numerically
# - try to classify the hopf bifurcation
plt.plot(betas[:21], volumes * 1e9, marker="o")
plt.plot(betas[:21], 1.51147e-5 ** 2 * curly_Vs[:21] / 9.81 * 1e13)
plt.xlabel("Aspect ratio $\\lambda$")
plt.ylabel("Particle volume (mm$^3$)")
plt.show()